# Cassava Leaf Disease Classification using tfrecords

This notebook utilizes train (original + augmented) and test tfrecords to read cassava leaf images for training using efficientNet architecture. Based on the training, the output image is classified as either a disease or a healthy leaf. Pre-trained weights are used from imagenet to train the model.

# Importing Packages

In [ ]:
import io
import os
import csv
import cv2

import pandas as pd
import numpy as np
from csv import writer

import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from functools import partial
import albumentations as A

from tensorflow import keras
from keras import layers
from keras.layers.core import Dense
from keras.models import Sequential

from tensorflow.keras.applications import EfficientNetB6

from numpy.random import seed
seed(3124)
from tensorflow.random import set_seed
set_seed(3124)

In [ ]:
BATCH_SIZE = 128
IMAGE_SIZE = [512, 512]
Epochs = 10
num_classes = 5
opt = keras.optimizers.Adam(learning_rate=0.001)

# Data Visualization

In [ ]:
csv_path ='../input/cassava-leaf-disease-classification/train.csv'
cld_data = pd.read_csv(csv_path)
cld_data.info()
sns.catplot(x='label', data=cld_data, kind='count')
plt.show()

So, there are about 21k leaf images in the dataset, each having label in range (0-4). The catplot shows that there are about 60% samples that belong to label-3 i.e.,leaves having "Cassava Mosaic Disease (CMD)" while the rest 4 classes collectively constitute 40% of the dataset.

# Data Augmentation

In [ ]:
def img_augumentation(image, no_transform, label, image_name):
  transforms = [
    A.VerticalFlip(p=0.7),
    A.RandomRotate90(p=0.7),
    A.RandomBrightness(p=0.8),
    A.RandomScale(p=0.8),
    A.RandomRotate90(p=0.5),
    A.RandomCrop(512,512),
    A.VerticalFlip(p=0.6),
    A.RandomRotate90(p=0.9),
    ]
    
  for i in range(no_transform):
    transform_aug = A.Compose([transforms[i]])
    transformed = transform_aug(image=image)
    transformed_image = transformed["image"]
    transformed_images.append(transformed_image)
    transformed_images_names.append(image_name+str(i)+".jpg")
    transformed_labels.append(label)
    
  

# Writing tfrecords for Augmented Data

In [ ]:

def _bytes_feature(value):
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() 
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def image_example(image_string, image_name, label):
  feature = {
        'target': _int64_feature(label),
        'image': _bytes_feature(image_string),
        'image_name': _bytes_feature(image_name),
  }
  tf_example = tf.train.Example(features=tf.train.Features(feature=feature))
  return tf_example

def write_record(image, image_name, label): 
  with tf.io.TFRecordWriter(saving_path) as writer:
    for img_count in range(len(image)):
      is_success, im_buf_arr = cv2.imencode(".jpg", image[img_count])
      image_string = im_buf_arr.tobytes()
      image_name_bytes = str.encode(image_name[img_count])
      tf_example = image_example(image_string, image_name_bytes, label[img_count])
      writer.write(tf_example.SerializeToString())

def write_csv(transformed_names, label):
  with open('./train.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    for img_count in range(len(transformed_names)):
      new_row = [transformed_names[img_count],label[img_count]]
      writer.writerow(new_row)

    

In [ ]:
csv_path ="../input/cassava-leaf-disease-classification/train.csv"
data = pd.read_csv(csv_path)
submission_data = data.to_csv('./train.csv', index = False)
aug_data = pd.read_csv('./train.csv')

if not os.path.exists('./train_tfrecords'):
  ! mkdir './train_tfrecords'

PATH = "./train_tfrecords/ld_train"
img_folder = "../input/cassava-leaf-disease-classification/train_images"
rec_counter = 15
labels = [0,1,2,3,4]
num_rec = 25
no_of_transformations = [8,4,4,0,4]
num_arr = [42,84,84,0,84]
num_images = 1344
start = np.zeros(5,int)
end =  np.zeros(5,int)

for record in range (num_rec):
  transformed_images = []
  transformed_images_names = []
  transformed_labels = [] 

  for label in range(len(labels)):
    end[label] = end[label] + num_arr[label]
    if labels[label]!=3:
      filter = data["label"]==labels[label] 
      _label = data.where(filter)
      _label = _label.dropna()
  
      for image_name in _label["image_id"][start[label]:end[label]]:  
        img_path = os.path.join(img_folder,image_name)
        img = plt.imread(img_path)
        img_augumentation(img, no_of_transformations[label], labels[label], image_name[0:-4])

  rec_counter = rec_counter+1
  for i in range(5):
        start[i] = end[i]
  saving_path = PATH + str(rec_counter) + "-" + str(num_images) + ".tfrec"
  write_record(transformed_images, transformed_images_names,transformed_labels) 
  write_csv(transformed_images_names,transformed_labels)
  
  


# Visualizing Augmented Data 

In [ ]:
csv_path ='./train.csv'
cld_data = pd.read_csv(csv_path)
cld_data.info()
sns.catplot(x='label', data=cld_data, kind='count')
plt.show()

By using augmentation,55k leaf images in total are obtained, each having label in range (0-4).

# Reading tfrecords
In the next step, to speed up the training process and effective utilization of avaialble memory, tfrecords are used to retrieve images, their names and labels with BATCHSIZE of 128. Code for reading tfrecords is taken from [here](https://keras.io/examples/keras_recipes/tfrecord/). 

**Data Splitting**

For 70:30 split of data, out of 41 tfrecords, 29 tfrecords are used for training purpose while the 12 are used as validation dataset.

In [ ]:
main_path='../input/cassava-leaf-disease-classification/train_tfrecords/ld_train'
aug_path="./train_tfrecords/ld_train"
train_Filenames = []
val_Filenames = []
test_Filenames = []

for record in range(0,12):
  if record < 10:
    train_path = main_path + "0" + str(record) + "-1338.tfrec"
  else:
    train_path = main_path + str(record) + "-1338.tfrec"
  train_Filenames.append(train_path) 

for record in range(24,41):
  train_path = aug_path + str(record) + "-1344.tfrec"
  train_Filenames.append(train_path) 

for record in range(16,24):
  val_path = aug_path + str(record) + "-1344.tfrec"
  val_Filenames.append(val_path)

for record in range(12,16):
  if record==15:
    val_path = main_path + str(record) + "-1327.tfrec"
  else:
    val_path = main_path + str(record) + "-1338.tfrec"
  val_Filenames.append(val_path)


test_path ="../input/cassava-leaf-disease-classification/test_tfrecords/ld_test00-1.tfrec"
test_Filenames.append(test_path)

print("Train TFRecord Files:", len(train_Filenames))
print("Validation TFRecord Files:", len(val_Filenames))
print("Test TFRecord Files:", len(test_Filenames))

**Training, Validation and Testing Dataset Creation**

In [ ]:
                
def read_tfrecord(example, labeled):
    tfrecord_format = (
        {
            "image_name": tf.io.FixedLenFeature([], tf.string),
            "image": tf.io.FixedLenFeature([], tf.string),
            "target": tf.io.FixedLenFeature([], tf.int64),
        }
        if labeled
        else {
            "image_name": tf.io.FixedLenFeature([], tf.string),
            "image": tf.io.FixedLenFeature([], tf.string),
        }
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example["image"])
    if labeled:
        label_plot = tf.cast(example["target"], tf.int32)        
        label = tf.one_hot(label_plot, depth=num_classes)
        return image, label_plot
    return image

def decode_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, IMAGE_SIZE)
    return image

AUTOTUNE = tf.data.experimental.AUTOTUNE

def load_dataset(filenames, labeled=True):
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(partial(read_tfrecord, labeled=labeled))
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames, labeled=labeled)
    dataset = dataset.shuffle(2048, reshuffle_each_iteration=False, seed=3124)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

train_dataset = get_dataset(train_Filenames)
val_dataset = get_dataset(val_Filenames)
test_dataset = get_dataset(test_Filenames, labeled=False)


After creating the datasets, first batch is loaded and images with their respective labels are displayed using below code.

In [ ]:
def show_batch(image_batch, label_batch):
    plt.figure(figsize=(12, 8))
    for n in range(12):
        ax = plt.subplot(3, 4, n + 1)
        plt.imshow(image_batch[n] / 255.0)
        if label_batch[n] == 4:
            plt.title("Healthy")
        elif label_batch[n] == 3:
            plt.title("Cassava Mosaic")
        elif label_batch[n] == 2:
            plt.title("Cassava Green Mottle")
        elif label_batch[n] == 1:
            plt.title("Cassava Brown Streak")
        else:
            plt.title("Cassava Bacterial Blight")
        plt.axis("off")

for image_batch, label_batch in train_dataset.take(1):
    show_batch(image_batch.numpy(), label_batch.numpy())

# Transfer learning (EfficientNet)
As the images retrieved using tfrecords have resolution 512x512, so EfficientNet version B6 is used that allows image resolution upto 528x528.

To determine the loss between labels and predictions, categorical cross entropy and adam oprtimizer is used. The model is trained for 10 epochs and validated on validation dataset. Pre-trained model is loaded and prediction is made.

In [ ]:

#model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics=['accuracy'])
#csv_logger = CSVLogger('training.log', separator=',', append=False)
#history = model.fit(train_dataset, epochs=100, validation_data=val_dataset, callbacks=[csv_logger])

#model loading
loaded_model = keras.models.load_model('../input/efficientnet-trained-model/trained_model.h5')
loaded_model.summary()


In [ ]:
#model history for 100 epochs
history = '../input/efficientnet-trained-model/training.log'
log_data = pd.read_csv(history, sep=',', engine='python')
acc = log_data["accuracy"]
loss = log_data["loss"]

val_acc = log_data["val_accuracy"]
val_loss = log_data["val_loss"]

print("Training Accuracy : {:.2f}".format(acc[9]*100),"%")
print("Validation Accuracy : {:.2f}".format(val_acc[9]*100),"%")


# Plot of Epochs vs Accuracy

In [ ]:
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label = 'Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

# Plot of Epochs vs Loss

In [ ]:
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label = 'Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

# Test Data Prediction

In [ ]:

predicted_label = np.argmax(loaded_model.predict(test_dataset))
print("Predicted Class :", predicted_label)
test_img = '2216849948.jpg'

data_dict = {'image_id':test_img,'label':predicted_label}

result = pd.DataFrame(data_dict, index = [1]) 
submission_data = result.to_csv('./submission.csv', index = False)
sub_csv = pd.read_csv('./submission.csv')
sub_csv